**REPOSITORIO UNSAAC**

- Alex Jesus Flores Taco   (20210833)



In [ ]:
# Cargando librerías o módulos pertinentes
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd

In [ ]:
# Creando una función que permite extraer todas las urls de las páginas que contienen
# las publicaciones encontradas en erespectivo repositorio
def all_pages(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag a que contiene el url base de las páginas que contienen las publicaciones
    pages = soup.find_all("a", {"class":"next-page-link"})
    # Extraer el url base de las páginas que contienen las publicaciones
    pages_soup = BeautifulSoup(str(pages), 'html.parser')
    a_tags = pages_soup.find_all('a') # find all anchor elements
    base_href = ["https://repositorio.unsaac.edu.pe/" + a_tag['href'] for a_tag in a_tags][0]
    # Crear lista de límites inferiores de cada url-página
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    lower_limit_max = (max([int(item) for item in matches]) // 20) * 20
    lower_limits = list(range(0, lower_limit_max + 20, 20))
    # Crear lista de urls de cada una de las páginas 
    hrefs_all = [re.sub(r"offset=\d+", "offset=" + str(pag), base_href) for pag in lower_limits]
    return hrefs_all

In [ ]:
all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')

In [ ]:
# Creando una función que permite extraer todas las urls de la tabla de datos de las
# publicaciones de una determina página
def links_scrapy(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url
    publication = soup.find_all("h4", {"class":"artifact-title"})
    # Extraer los url de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a') # find all anchor elements
    hrefs = ["https://repositorio.unsaac.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

In [ ]:
links_scrapy('https://repositorio.unsaac.edu.pe/browse?type=dateissued')

In [ ]:
### TENER CUIDADO CON AL EJECUCIOND E ESTE CODIGO SON 5769 TESIS Y MI LAPOTP NO AGUANTA

# Extracción de todas las urls de la tabla de datos de todas las
# publicaciones del repositorio
# NOTA: no se recomienda ejecutar esta celda, ya que toma demasiado tiempo obtener una salida
every_page = all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page]))

In [ ]:
# EJEMPLO DE EJECUCIÓN
# En este ejemplo solo se considera las dos primeras páginas del repositorio
every_page = all_pages('https://repositorio.unsaac.edu.pe/browse?type=dateissued')
publications_links = list(itertools.chain(*[links_scrapy(url) for url in every_page[0:2]]))

In [ ]:
publications_links